In [41]:
import sys
import os


In [42]:
from numpy import dtype
import pandas as pd

In [43]:
data = pd.read_excel('top_articles_by_category_20250917_20250923.xlsx')


## Define common functions

In [44]:
def cast_types(data):
    data['Publication Date'] = pd.to_datetime(data['Publication Date'], errors='coerce')
    return data

def filter_last_week(data):
    # 'Publication Date' is a dtype datetime64[ns]
    last_week = data[data['Publication Date'] >= '2025-09-10']
    return last_week

def top_n_by_category_last_week(data, n=3, metric='screenPageViews'):
    last_week = filter_last_week(data)
    out = {}
    for category, group in last_week.groupby('Categoria'):
        top_n = group.nlargest(n, metric)
        out[category] = top_n
    return out

def top_n_by_category(data, n=3, metric='screenPageViews', last_week=False):
    if last_week:
        data = filter_last_week(data)
    out = {}
    for category, group in data.groupby('Categoria'):
        top_n = group.nlargest(n, metric)
        out[category] = top_n
    return out 

def top_n(data, n=3, metric='screenPageViews', last_week=False):
    if last_week:
        data = filter_last_week(data)
    top_n = data.nlargest(n, metric)
    return top_n

def print_top_articles(data):
    # For each top article in the category, print Title and Author between parentheses
    for index, row in data.iterrows():
        print(f"{row['Title']} ({row['Author']})")

data = cast_types(data)
top_by_views = top_n_by_category(data, n=3)


## Set pandas options

In [45]:
# Pandas remove limit of title display
pd.set_option('display.max_colwidth', None)

## Absolute Top

In [46]:
top_by_views = top_n(data, n=20, metric='screenPageViews', last_week=False)
top_by_views[['Title', 'Author', 'Categoria', 'screenPageViews', 'averageSessionDuration', 'engagementRate', 'Publication Date']]

,Title,Author,Categoria,screenPageViews,averageSessionDuration,engagementRate,Publication Date
0,‘Beauty in Black 3’ si farà? Cosa sappiamo al momento,Enrico Ziliani,Si farà,577,60.708252,0.578313,2025-09-14
1,‘Uno splendido errore 3’: iniziano le riprese della terza stagione,Gaia Trentin,Si farà,522,35.588992,0.495202,2025-09-05
2,‘Countdown’: la serie Prime Video avrà una seconda stagione?,Valerio Fidanza,Serie TV,305,63.596024,0.560897,2025-06-26
4,"‘In Re Minore’, quando il cinema mette in scena verità, dolore e rinascita",Ylenia Politano,Speciali e Magazine,296,195.453078,0.417526,2025-09-17
3,‘Tienimi presente’ in anteprima al Festival del Cinema Di Roma,Mattia Barichello,Speciali e Magazine,208,94.096170,0.460317,2025-09-19
5,‘KPop Demon Hunters 2’: Netflix pronto a parlare del sequel,Aurora Fazi,News,139,42.646418,0.503546,2025-08-27
8,Festa del Cinema di Roma 2025: annunciato il Programma,Ilaria Petroni,Festival di Cinema,125,78.135223,0.629032,2025-09-19
7,‘UFO Robot Goldrake’ torna in TV dopo 50 anni: ecco dove vederlo,Eleonora Zanardi,Animazione,123,106.821368,0.652893,2025-09-18
6,Il finale di ‘Diario dei miei due di picche’: ci sarà una stagione 2?,Irene Casula,Si farà,121,82.843786,0.650407,2025-09-16
14,‘Mi Faccio di Rock’n’Roll’: un piccolo cult del nostro cinema,Emanuele Grimaldi,Live Streaming On Demand,121,167.360458,0.574257,2025-09-18


## Top by Category

In [47]:
category = 'Altro'
top_by_views = top_n_by_category(data, n=5)
print_top_articles(top_by_views[category])
top_by_views[category][['Title', 'screenPageViews', 'Publication Date', 'Categoria', 'Author']]


‘Tutto quello che resta di te’ – Il cinema come atto di scelta (Lavinia Colanzi)


,Title,screenPageViews,Publication Date,Categoria,Author
34,‘Tutto quello che resta di te’ – Il cinema come atto di scelta,90,2025-09-14,Altro,Lavinia Colanzi


In [48]:
def format_ranking(data):
    """
    Format top articles for ranking as requested.
    Each item: ‘Title’ (Author) [Categoria]
    """
    lines = ["\U0001F3AA Classifica per coinvolgimento:", ""]
    for idx, row in data.iterrows():
        title = row.get('Title', '(titolo mancante)')
        author = row.get('Author', 'Anonimo')
        categoria = row.get('Categoria', '')
        lines.append(f"{title} ({author}) [{categoria}]")
    return "\n".join(lines)

metric = 'engagementRate'  # or 'averageSessionDuration', etc.
n = 20
top_by_engagement = top_n(data, n=n, metric=metric, last_week=True)
print(format_ranking(top_by_engagement))
top_by_engagement[['Title', 'Author', 'Categoria', metric, 'averageSessionDuration', 'screenPageViews', 'Publication Date']]

🎪 Classifica per coinvolgimento:

Il Sundance dice addio a Robert Redford: fondatore, mentore, amico (Greta Wieth) [Speciali e Magazine]
Barbra Streisand e Robert Redford: una storia di cinema e amicizia (Enrico Ziliani) [Anticipazioni]
‘The Life of Chuck’: distopie esistenziali aspettando l’Apocalisse (Cristina Locuratolo) [Recensioni]
‘Le maledizioni’: il thriller politico argentino che approda su Netflix (Lorenzo Zanotti) [Serie TV]
‘Super happy forever’. Che vita fanno le cose che abbiamo perso? (Gioia Zurlo) [Recensioni]
‘Jane Austen ha stravolto la mia vita’: una storia poco austeniana (Ermelinda Bisantis) [Recensioni]
Festa del cinema di Roma 2025 i Film italiani in concorso (Sandra Orlando) [Festival di Cinema]
‘Die My Love’ con Jennifer Lawrence e Robert Pattinson: debutto a Roma e presto nelle sale italiane (Giacomo Mosca) [News]
‘Alarum’: Un cast dai grandi nomi per un film che non convince (Emanuele Grimaldi) [Live Streaming On Demand]
‘Una battaglia dopo l’altra’, l’ultimo

,Title,Author,Categoria,engagementRate,averageSessionDuration,screenPageViews,Publication Date
67,"Il Sundance dice addio a Robert Redford: fondatore, mentore, amico",Greta Wieth,Speciali e Magazine,0.818182,166.375904,34,2025-09-18
15,Barbra Streisand e Robert Redford: una storia di cinema e amicizia,Enrico Ziliani,Anticipazioni,0.795455,100.525044,91,2025-09-18
82,‘The Life of Chuck’: distopie esistenziali aspettando l’Apocalisse,Cristina Locuratolo,Recensioni,0.750000,82.577275,33,2025-09-23
9,‘Le maledizioni’: il thriller politico argentino che approda su Netflix,Lorenzo Zanotti,Serie TV,0.747664,58.873560,108,2025-09-17
78,‘Super happy forever’. Che vita fanno le cose che abbiamo perso?,Gioia Zurlo,Recensioni,0.741935,97.214388,37,2025-09-18
24,‘Jane Austen ha stravolto la mia vita’: una storia poco austeniana,Ermelinda Bisantis,Recensioni,0.740260,114.792526,75,2025-09-17
35,Festa del cinema di Roma 2025 i Film italiani in concorso,Sandra Orlando,Festival di Cinema,0.732143,66.396515,62,2025-09-19
81,‘Die My Love’ con Jennifer Lawrence e Robert Pattinson: debutto a Roma e presto nelle sale italiane,Giacomo Mosca,News,0.724138,88.506807,31,2025-09-19
54,‘Alarum’: Un cast dai grandi nomi per un film che non convince,Emanuele Grimaldi,Live Streaming On Demand,0.717949,49.914008,39,2025-09-22
61,"‘Una battaglia dopo l’altra’, l’ultimo film di Paul Thomas Anderson",Debora Lai,Anticipazioni,0.710526,202.801234,38,2025-09-16


## Analisi per categoria

In [49]:
views_by_category = data.groupby('Categoria')['screenPageViews'].sum() 
(views_by_category * 100 / views_by_category.sum()).sort_values(ascending=False)


Categoria
Si farà                     26.032190
Serie TV                    16.389083
Speciali e Magazine         12.806158
Festival di Cinema           9.951015
News                         8.047586
Recensioni                   7.865640
Anticipazioni                7.599720
Trailers                     3.317005
Live Streaming On Demand     2.827152
Animazione                   1.721484
Interviste                   1.413576
Altro                        1.259622
Rubriche                     0.769769
Name: screenPageViews, dtype: float64

In [50]:
CATEGORIES = {
    "News": {"latest-news", "focus-italia"},
    "Anticipazioni": {"anticipazioni"},
    "Recensioni": {"review", "netflix-film", "sky-film", "disney-film", "mubi", "mubi-film",
                    "approfondimenti", "streaming"},
    "In Sala": {"in-sala"},
    "Cult Movies": {"cult-movie"},
    "Animazione": {"animazione", "animazione/anime"},
    "Approfondimento": {"approfondimento"},
    "Festival di Cinema": {"festival-di-cinema"},
    "Trailers": {"trailers"},
    "Serie TV": {"serie-tv", "netflix-serie-tv", "prime-video-serietv", "sky-serie-tv",
                 "disney-serietv", "paramount-serie-tv", "appletv-serietv", "tim-vision-serie-tv"},
    "Guide e Film da Vedere": {"film-da-vedere"},
    "Speciali e Magazine": {"magazine-2", "taxidrivers-magazine"},
    "Live Streaming On Demand": {"live-streaming-on-demand"},
    "Rubriche": {"rubriche"},
    "Interviste": {"interviews"}
}

path = "/435878/review/in-sala/jane-austen-ha-stravolto-la-mia-vita-una-storia-poco-austeniana.html"

subpaths = path.split('/')
# Category mapping without using subpaths
for category, keywords in CATEGORIES.items():
    for keyword in keywords:
        if keyword in subpaths:
            # Special case: category mixed with "in-sala"
            if "anticipazioni" in subpaths:
                print("Anticipazioni")
            if category == "Recensioni" and "in-sala" in subpaths:
                print("Recensioni / In Sala")
                break
            if category == "Trailers" and "in-sala" in subpaths:
                print("Trailers / In Sala")
            break
            print(category)


Recensioni / In Sala
